In [3]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import math
from scipy.optimize import minimize
from scipy.optimize import leastsq

In [4]:
from poly_func import getMatrixP

Implement the modified miniziation function to obtain coefficients of polynomial equations

In [36]:
params = ['x1', 'x2']
order = 2
nscans = 12
k1 = 0.1
k2 = 5
k11 = 0.01
k22 = 0.05
k12 = 0
norm_val = 10
xmin = 1
xmax = 2

In [6]:
def generate_linear_data(nscans, p_err=0.01):
    dx1 = np.random.uniform(xmin, xmax, nscans)
    dx2 = np.random.uniform(xmin, xmax, nscans)
    norm = np.array([norm_val]* nscans)
    
    data = norm + k1*dx1 + k2*dx2
    error = data * np.random.normal(loc=0, scale=p_err, size=nscans)
    input_matrix = np.array([dx1, dx2])
    
#     return (input_matrix, data, error)
    return (input_matrix, data+error, error)

In [7]:
def generate_quad_data(nscans, p_err=0.01):
    dx1 = np.random.uniform(xmin, xmax, nscans)
    dx2 = np.random.uniform(xmin, xmax, nscans)
    norm = np.array([norm_val]* nscans)
    
    data = norm + k1*dx1 + k2*dx2 + k11*dx1*dx1 + k22*dx2*dx2 + k12*dx1*dx2
    error = data * np.random.normal(loc=0, scale=p_err, size=nscans)
    input_matrix = np.array([dx1, dx2])
    
    return (input_matrix, data+error, error)

In [8]:
generate_data = generate_quad_data

In [9]:
def get_N_coeffienct(n_x, order):
    N = 1
    for io in range(1, order+1):
        multi = 1
        for j in range(io):
            multi *= n_x + j
        multi /= math.factorial(io)
        N+= multi
    return N

In [10]:
get_N_coeffienct(11, 3)

364.0

In [11]:
def get_N_entry(n_x, order):
    multi = 1
    for j in range(order):
        multi *= n_x + j
    multi /= math.factorial(order)

    return multi

In [12]:
def get_objective_func(P, data, error):
    """The objective function to be minized"""
    nscans = P.shape[0]
    def objective(X):
        return sum( (np.matmul(P, X) - data)**2/error**2 )/nscans
    
    return objective

In [13]:
def get_objective_func_noError(P, data, error):
    """The objective function to be minized"""
    nscans = P.shape[0]
    def objective(X):
        return sum( (np.matmul(P, X) - data)**2 )/nscans
    
    return objective

In [47]:
nscans = 20
in_matrix, data, error = generate_data(nscans, 0.05)
test_matrix, test_data, test_error = generate_data(nscans, 0.05)

In [48]:
P = getMatrixP(in_matrix, order)

In [49]:
P.shape

(20, 6)

In [50]:
objective = get_objective_func(P, data, error)
objective_no_error = get_objective_func_noError(P, data, error)
obj_test = get_objective_func_noError(P, test_data, error)

In [51]:
X0 = [1]*int(get_N_coeffienct(len(params), order))

In [52]:
X0

[1, 1, 1, 1, 1, 1]

In [53]:
objective(X0)

6250.378347062318

In [54]:
res = minimize(
    objective, X0, method='nelder-mead', 
    options={'xtol':1e-8, 'disp':True}
)
print(res.success)

False


In [55]:
XP = np.matmul(np.linalg.pinv(P), data)

In [56]:
XP

array([  8.95497325, -17.08963931,  24.41178334,   6.3236684 ,
        -3.21090224,  -4.45380914])

In [21]:
from iminuit import Minuit

In [26]:
def obj_fnc(P, data, error):
    """The objective function to be minized"""
    nscans = P.shape[0]
    def objective(x0, x1, x2, x3, x4, x5):
        return (P[0][0]*x0 +  P[0][1]*x1 + P[0][2]*x2 + P[0][3]*x3 +  P[0][4]*x4 + P[0][5]*x5 - data[0])**2 / error[0]**2 \
    + (P[1][0]*x0 +  P[1][1]*x1 + P[1][2]*x2 + P[1][3]*x3 +  P[1][4]*x4 + P[1][5]*x5 - data[1])**2 / error[1]**2 \
    + (P[2][0]*x0 +  P[2][1]*x1 + P[2][2]*x2 + P[2][3]*x3 +  P[2][4]*x4 + P[2][5]*x5 - data[2])**2 / error[2]**2 \
    + (P[3][0]*x0 +  P[3][1]*x1 + P[3][2]*x2 + P[3][3]*x3 +  P[3][4]*x4 + P[3][5]*x5 - data[3])**2 / error[3]**2 \
    + (P[4][0]*x0 +  P[4][1]*x1 + P[4][2]*x2 + P[4][3]*x3 +  P[4][4]*x4 + P[4][5]*x5 - data[4])**2 / error[4]**2 \
    + (P[5][0]*x0 +  P[5][1]*x1 + P[5][2]*x2 + P[5][3]*x3 +  P[5][4]*x4 + P[5][5]*x5 - data[5])**2 / error[5]**2
    
    return objective

In [27]:
obj = obj_fnc(P, data, error)


In [30]:
m0 = Minuit(obj)

m0.migrad()  # run optimiser

/global/homes/x/xju/.conda/envs/uproot/lib/python3.7/site-packages/ipykernel_launcher.py:1: InitialParamWarning: Parameter x0 does not have neither initial value nor limits.
  """Entry point for launching an IPython kernel.
/global/homes/x/xju/.conda/envs/uproot/lib/python3.7/site-packages/ipykernel_launcher.py:1: InitialParamWarning: Parameter x1 does not have neither initial value nor limits.
  """Entry point for launching an IPython kernel.
/global/homes/x/xju/.conda/envs/uproot/lib/python3.7/site-packages/ipykernel_launcher.py:1: InitialParamWarning: Parameter x2 does not have neither initial value nor limits.
  """Entry point for launching an IPython kernel.
/global/homes/x/xju/.conda/envs/uproot/lib/python3.7/site-packages/ipykernel_launcher.py:1: InitialParamWarning: Parameter x3 does not have neither initial value nor limits.
  """Entry point for launching an IPython kernel.
/global/homes/x/xju/.conda/envs/uproot/lib/python3.7/site-packages/ipykernel_launcher.py:1: InitialParam

------------------------------------------------------------------
| FCN = 1.017e-09               |     Ncalls=323 (323 total)     |
| EDM = 9.61e-10 (Goal: 0.0002) |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   False   |   False   |  True  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | x0   |   -207    |     4     |            |            |         |         |       |
| 1 | x1   |    30     |     4     |            |            |         |         |       |
| 2 | x2   |   269.4   |    3.5    |            |            |         |         |       |
| 3 | x3   |   -11.8   |    1.8    |            |            |         |         |       |
| 4 | x4   |    9.4    |    2.5    |            |            |         |         |       |
| 5 | x5   |   -95.3   |    1.9    |            |            |         |         |       |
------------------------------------------------------------------------------------------

In [32]:
m0.minos()

------------------------------------------------------------------------------------------------------------------------------------------------------------
|          |          x0           |          x1           |          x2           |          x3           |          x4           |          x5           |
------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Error   |    -4     |     4     |    -50    |    50     |   -3.5    |    3.5    |    -10    |    10     |   -2.5    |    2.5    |   -1.9    |    1.9    |
|  Valid   |   False   |   False   |   True    |   True    |   False   |   False   |   True    |   True    |   False   |   False   |   False   |   False   |
| At Limit |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |
| Max FCN  |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |
| New Min  |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |   False   |
------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
XP

array([-206.74705589,   29.79897052,  269.35601896,  -11.80954693,
          9.36988264,  -95.34513421])

In [19]:
print(objective(res.x))
print(res.message)

0.0053252531074395384
Maximum number of function evaluations has been exceeded.


In [20]:
res_no_error = minimize(
    objective_no_error, X0, method='nelder-mead', 
    options={'xtol':1e-8, 'disp':True}
)
print(res_no_error.success)

False


In [21]:
print("Truth: " + " ".join("{:.2f}".format(xx) for xx in [norm_val, k1, k2, k11, k22, k12]))
X = res.x.tolist()
X2 = res_no_error.x.tolist()
XP = np.matmul(np.linalg.pinv(P), data)
print("With error: " + " ".join(["{:.2f}".format(xx) for xx in X]))
print("No error: " + " ".join(["{:.2f}".format(xx) for xx in X2]))
print("Psudo-Inv: " + " ".join(["{:.2f}".format(xx) for xx in XP]))
print(objective_no_error(X))
print(objective_no_error(X2))
print(objective_no_error(XP))
print(obj_test(X))
print(obj_test(X2))
print(obj_test(XP))

Truth: 10.00 0.10 5.00 0.01 0.05 0.00
With error: -17.43 40.67 7.01 -1.93 -23.06 8.56
No error: 2.40 38.83 -20.59 11.68 -46.15 27.79
Psudo-Inv: -18.13 43.53 5.11 -0.76 -26.95 10.87
0.007967988469502621
0.3201807613441016
7.182315588402238e-27
7.907813686169796
8.929509807675613
8.224947324512444


In [22]:
def study_scans(nscans, p_err=0.01):
    in_matrix, data, error = generate_data(nscans, p_err)
    test_data = generate_data(nscans, p_err)[1]

    P = getMatrixP(in_matrix, order)
    objective = get_objective_func(P, data, error)
    objective_no_error = get_objective_func_noError(P, data, error)
    obj_test = get_objective_func_noError(P, test_data, error)
    
    X0 = [1]*int(get_N_coeffienct(len(params), order))
    res = minimize(
        objective, X0, method='nelder-mead', 
        options={'xtol':1e-8, 'disp':False}
    )

    res_no_error = minimize(
        objective_no_error, X0, method='nelder-mead', 
        options={'xtol':1e-8, 'disp':False}
    )
    
    X = res.x.tolist()
    X2 = res_no_error.x.tolist()    
    XP = np.matmul(np.linalg.pinv(P), data)

    return obj_test(X), obj_test(X2), obj_test(XP)

In [23]:
def plot(res_list, label_list, out_name):
    res1 = res_list[0]
    mean = np.mean(res1)
    std = np.std(res1)
    hist_x_max = mean + 3*std
    hist_x_min = mean - 3*std
    nbins = 50
    for res, label in zip(res_list, label_list):
        plt.hist(res, bins=nbins, range=(hist_x_min, hist_x_max),
                 histtype='step', label=label, lw=2)

    plt.legend()
    plt.savefig(out_name)

In [24]:
def stats(results):
    for ir in results:
        print(np.mean(ir), np.std(ir))

In [25]:
labels = ['with error', 'no error', 'pseudo-inverse']

In [26]:
res_with_error100 = []
res_no_error100 = []
res_pinv_error100 = []
for i in range(5000):
    with_error, no_err, pinv_err = study_scans(30, 0.05)
    res_with_error100.append(with_error)
    res_no_error100.append(no_err)
    res_pinv_error100.append(pinv_err)

res100 = [res_with_error100, res_no_error100, res_pinv_error100]

plot(res100, labels)
stats(res100)

TypeError: plot() missing 1 required positional argument: 'out_name'

In [ ]:
res_with_error100 = []
res_no_error100 = []
res_pinv_error100 = []
for i in range(5000):
    with_error, no_err, pinv_err = study_scans(30, 0.05)
    res_with_error100.append(with_error)
    res_no_error100.append(no_err)
    res_pinv_error100.append(pinv_err)

res100 = [res_with_error100, res_no_error100, res_pinv_error100]

plot(res100, labels, 'test_5pecent_error.pdf')
stats(res100)

In [ ]:
results = []
results.append([])
results.append([])
results.append([])

for i in range(5000):
    with_error, no_err, pinv_err = study_scans(30, 0.10)
    results[0].append(with_error)
    results[1].append(no_err)
    results[2].append(pinv_err)

plot(results, labels)
stats(results)

In [ ]:
70*1000000/25000

In [ ]:
2800/32